# Ayudantía Virtual: Metaclases

Autores originales = **Sebastián Guerra y Enzo Tamburini**


## Actividad 07 2016-2

 

En esta ayudantía virtual realizaremos [esta actividad](https://github.com/IIC2233-2016-02/Syllabus/blob/master/Actividades/AC07/Enunciado.pdf), la cual traía consigo un archivo _main.py_ cuyo contenido está al final de esta ayudantía :)

A continuación hay un extracto del _main.py_ con la definición de las clases:

In [ ]:
# Solo modificar para agregar metaclass=*
class Boss():
    def __init__(self, organization, *args, **kwargs):
        self.organization = organization

    def __repr__(self):
        return 'Boss: {0.name} {0.last_name}'.format(self)


class Worker():
    def __init__(self, organization, *args, **kwargs):
        self.organization = organization

    def __repr__(self):
        return 'Worker: {0.name} {0.last_name}'.format(self)


class Organization():

    def __init__(self, name):
        self.name = name
        self.boss = None
        self.members = list()

    def __repr__(self):
        return 'Organizacion: {}'.format(self.name)

    def pick_one_worker(self):
        return random.choice(self.members)


A esto sólo hay que agregarle la sentencia `(metaclass = MetaClaseDeseada)`

## Instrucciones

Dentro del archivo _main.py_ descargado junto a este enunciado encontrará tres clases; `Boss`, `Worker` y `Organization`. Dichas clases se encuentran funcionalmente incompletas y es deber suyo corregirlas, sólo
mediante el **uso de metaclases**.
Deberá crear las metaclases `MetaOrganization` y `MetaPerson` para posteriormente poder aplicarlas
sobre cualquier clase, en particular, `MetaOrganization` sobre `Organization` y `MetaPerson` sobre `Boss` y `Worker`
para completar el funcionamiento del programa.

A partir de ahora el enunciado llamará **Organización** a cualquier clase con metaclase `MetaOrganization`
y **Persona** a cualquier clase con metaclase `MetaPerson`.


## Requerimientos

> Cualquier clase `Organizacion` debe cumplir:

> - Poseer un método para listar a sus miembros.

Primero declaramos la metaclase, para esto recordar que para crear una, se debe heredar de `type`. Luego, para resolver el primer requerimiento, debemos definir una función dentro de `__new__` y asignarla en el diccionario de esta. 

In [3]:
class MetaOrganization(type):
    def __new__(meta, name, base, clsdict):
        
        def see_members(self):
            for i, miembro in enumerate(self.members):
                print('Miembro numero {0}: {1}\n'.format(i, miembro))
                
        clsdict['see_members'] = see_members
        
        return super().__new__(meta, name, base, clsdict)

> - Poseer un método para reemplazar al jefe actual por otro.

In [4]:
class MetaOrganization(type):
    def __new__(meta, name, base, clsdict):
        def see_members(self):
            for i, miembro in enumerate(self.members):
                print('Miembro numero {0}: {1}\n'.format(i, miembro))
        
        def replace_boss(self, new_boss):
            self.boss = new_boss
        
        clsdict['see_members'] = see_members
        clsdict['replace_boss'] = replace_boss
        
        return super().__new__(meta, name, base, clsdict)

> - Cuando **se llama** a una de sus instancias, se imprime la información de la instancia en cuestión; **nombre**, **jefe** y **cantidad de miembros**.

---

**OJO:** Aquí hay que hacer una útil distinción. Vamos a sobreescribir el `__call__` de la clase creada. Este método es **diferente** al de la metaclase. Para ver la diferencia, un ejemplo: 

In [5]:
class MetaEjemplo(type):
    def __call__(cls, *args, **kwargs):
        print('Siempre me imprimiré! porque soy parte de la instanciación de la clase >:)')
        return super().__call__(*args, **kwargs)
    
class Ejemplin(metaclass=MetaEjemplo):
    def __call__(self, *args, **kwargs):
        print('Sólo me imprimo cuando ejecutan la instancia! :B')
        print('Y no retorno :P (aunque podría..)')
        
ej1 = Ejemplin() # Al llamar a una clase (instanciar un objeto), se llama al método __call__ de su metaclase.
ej1()            # Al llamar a una instancia, se llama al método __call__ de su clase.

Siempre me imprimiré! porque soy parte de la metaclase y por ende, de la creación de la clase >:)
Sólo me imprimo cuando ejecutan la instancia! :B
Y no retorno :P


Ambos métodos aunque se llamen igual, hacen cosas diferentes. Uno afecta directamente a la creación del objeto de la clase `cls` y que por lo tanto **debe** retornar algo (aunque sea `None`), mientras que el otro se pone en el caso de que se haga una llamada a la instancia (en el ejemplo: `ej1()`) y que por lo tanto, **puede, o no,** retornar algo.

---

Retomando la actividad...

In [6]:
class MetaOrganization(type):
    def __new__(meta, name, base, clsdict):
        def see_members(self):
            for i, miembro in enumerate(self.members):
                print('Miembro numero {0}: {1}\n'.format(i, miembro))
        
        def replace_boss(self, new_boss):
            self.boss = new_boss
            
        def call(self, *args, **kwargs):
            print('Nombre de la Organizacion: {0}\nJefe: {1}\nNumero de empleados: {2}\n'.format(self.name, self.boss, len(self.members)))

        clsdict['__call__'] = call
        clsdict['see_members'] = see_members
        clsdict['replace_boss'] = replace_boss
        
        return super().__new__(meta, name, base, clsdict)

> - Si al intentar instanciar una organización de clase `Organizacion` y el nombre se encuentra ocupado por alguna otra instancia de **dicha** clase `Organizacion`, la instanciación se verá denegada, retornando `None` en vez de la instancia esperada.

Para este requerimiento, es necesario irse al método `__call__` ya que es necesario uno de los argumentos que se le pasa a la instancia de la clase al momento de ser creada (el nombre de la organización). También notar que se guardarán los nombres de las instancias creadas.

In [7]:
class MetaOrganization(type):
    
    instances = dict()
    
    def __new__(meta, name, base, clsdict):
        def see_members(self):
            for i, miembro in enumerate(self.members):
                print('Miembro numero {0}: {1}\n'.format(i, miembro))
        
        def replace_boss(self, new_boss):
            self.boss = new_boss
            
        def call(self, *args, **kwargs):
            print('Nombre de la Organizacion: {0}\nJefe: {1}\nNumero de empleados: {2}\n'.format(self.name, self.boss, len(self.members)))

        clsdict['__call__'] = call
        clsdict['see_members'] = see_members
        clsdict['replace_boss'] = replace_boss
        
        return super().__new__(meta, name, base, clsdict)
    
    def __call__(cls, *args, **kwargs):
        if cls.__name__ == 'Organization':
            if args[0] in MetaOrganization.instances.keys():
                return None
            instance = super().__call__(*args, **kwargs)
            MetaOrganization.instances[args[0]] = instance
        return instance

> Cualquier clase `Persona` debe cumplir: 

> - Cuando se instancia una persona a partir de una clase `Persona` esta recibe un nombre, apellido y edad aleatorios.

Debido a que cada vez que se crea una nueva instancia de una `Persona` esta debe tener su propio nombre, apellido y edad, estos atributos deben ser creados a nivel del `__call__` de la metaclase. Si se aplican en el `__new__` **cada** instancia tendrá los mismos valores!!!

In [8]:
class MetaPerson(type):
    def __call__(cls, *args, **kwargs):
        instance = super().__call__(*args, **kwargs)
        
        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

> - El atributo `organizacion` de la clase persona debe guardar el nombre de la organización, **no la instancia organización**.

Para este requerimiento, debemos alterar los argumentos que recibe el método `__call__` de la metaclase. Recordemos que `*args` y `**kwargs` son los argumentos usados al instanciar una `Persona`. Como el primer argumento es el que se guarda en el atributo `organizacion`, es este el que debemos alterar.

In [9]:
class MetaPerson(type):
    def __call__(cls, *args, **kwargs):
        orga = args[0]        
        instance = super().__call__(orga.name, *args[1:], **kwargs)
        
        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

> - Si el nombre de la clase `Persona` contiene la palabra "Boss" se le considerará como una clase `Jefe`. Si contiene la palabra "Worker" se le considerará como una clase `Trabajador`.
> - Al momento de intentar instanciar una persona a partir de una clase `Persona` se debe cumplir:
>  * Si la organización **no posee** un jefe y la persona **no es un jefe**, no se verá instanciada, retornando `None`.
>  * Si la organización **no posee** un jefe y la persona **es un jefe**, se verá instanciada. Además, la organización le adoptará como jefe, otorgándole el método `add_members` para añadir personas a la organización. 
>  * Si la organización **posee** un jefe y la persona **no es un jefe**, será instanciada, pero no añadida a la organización necesariamente. Aceptar a dicha persona es deber del jefe.
>  * Si la organización **posee** un jefe y la persona **es un jefe**, se reemplazará al jefe actual de la organización por el nuevo jefe mediante el método de la organización correspondiente.
> * Considere que el método `add_members` recibe como parámetro un miembro a agregar a la organización. Este método debe agregarlos a la misma organización que pertenece el jefe.

Usaremos dos variables booleanas llamada `es_jefe` y `tiene_jefe`, las cuales indicarán si una `Persona` es jefe o no y si la organización tiene jefe respectivamente. Para el primer requerimiento,

In [10]:
class MetaPerson(type):
    def __call__(cls, *args, **kwargs):
        orga = args[0]        
        es_jefe = "Boss" in cls.__name__
        tiene_jefe = orga.boss is not None
        if not es_jefe and not tiene_jefe:
            return None
        instance = super().__call__(orga.name, *args[1:], **kwargs)
    
        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

Para el segundo requerimiento, se debe agregar un método para los jefes que permita agregar miembros a la organización que dirige.

In [11]:
class MetaPerson(type):
    def __new__(meta, name, bases, clsdict):
        if "Boss" in name:
            def add_member_method(self, member):
                MetaOrganization.instances[self.organization].members.append(member)
            clsdict["add_member"] = add_member_method
        
        return super().__new__(meta, name, bases, clsdict)
        
    def __call__(cls, *args, **kwargs):
        orga = args[0]        
        es_jefe = "Boss" in cls.__name__
        tiene_jefe = orga.boss
        if not es_jefe and not tiene_jefe:
            return None
        instance = super().__call__(orga.name, *args[1:], **kwargs)
        if es_jefe and not tiene_jefe:
            orga.boss = instance

        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

Para el tercer requerimiento solo se retorna tal y como ya se hace. Para el cuarto, se modifica el jefe actual de la organización.

In [12]:
class MetaPerson(type):
    def __new__(meta, name, bases, clsdict):
        if "Boss" in name:
            def add_member_method(self, member):
                MetaOrganization.instances[self.organization].members.append(member)
            clsdict["add_member"] = add_member_method
        
        return super().__new__(meta, name, bases, clsdict)
        
    def __call__(cls, *args, **kwargs):
        orga = args[0]        
        es_jefe = "Boss" in cls.__name__
        tiene_jefe = orga.boss
        if not es_jefe and not tiene_jefe:
            return None
        instance = super().__call__(orga.name, *args[1:], **kwargs)
        if es_jefe and not tiene_jefe:
            orga.boss = instance
        elif es_jefe and tiene_jefe:
            orga.replace_boss(instance)

        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

> - Cualquier clase `Persona` considerada Jefe deberá poseer un método para dar órdenes `to order`. Puede ser simulado mediante una impresión en pantalla.
> - Cualquier clase `Persona` considerada Trabajador deberá poseer un método para trabajar `to work`. Puede ser simulado mediante una impresión en pantalla.

In [13]:
class MetaPerson(type):
    def __new__(meta, name, bases, clsdict):
        if "Boss" in name:
            def add_member_method(self, member):
                MetaOrganization.instances[self.organization].members.append(member)
            clsdict["add_member"] = add_member_method
            
            def to_orden_method(self):
                print("Estoy mandando!")
            clsdict["order"] = to_orden_method
            
        elif "Worker" in name:
            def to_work_method(self):
                print("Estoy trabajando!")
            clsdict["to_work"] = to_work_method
        
        return super().__new__(meta, name, bases, clsdict)
        
    def __call__(cls, *args, **kwargs):
        orga = args[0]        
        es_jefe = "Boss" in cls.__name__
        tiene_jefe = orga.boss
        if not es_jefe and not tiene_jefe:
            return None
        instance = super().__call__(orga.name, *args[1:], **kwargs)
        if es_jefe and not tiene_jefe:
            orga.boss = instance
        elif es_jefe and tiene_jefe:
            orga.replace_boss(instance)

        setattr(instance, "name", random.choice(name_list))
        setattr(instance, "last_name", random.choice(lastname_list))
        setattr(instance, "age", random.randint(18, 65))
        
        return instance

---

### _main.py_
Contenido del archivo _main.py_. Ahora que se crearon completamente las Metaclases pedidas, el código funciona a la perfección.

In [14]:
import random

name_list = ['Alfonso', 'Benito', 'Alfredo', 'Geronimo', 'Peter', 'Jack',
             'Simon', 'Jaime', 'Bego', 'Francisca', 'Maida', 'Clara', 'Rocio',
             'Sofia', 'Belen', 'Fausto', 'Juan', 'Miguel', 'Mariana',
             'Fernanda', 'Constanza', 'Valentina', 'Tomas']

lastname_list = ['Fernández', 'Rodríguez', 'González', 'García', 'López',
                 'Martínez', 'Pérez', 'Álvarez', 'Gómez', 'Sánchez',
                 'Díaz', 'Vásquez', 'Castro', 'Romero', 'Suárez']


# Solo modificar para agregar metaclass=*
class Boss(metaclass=MetaPerson):
    def __init__(self, organization, *args, **kwargs):
        self.organization = organization

    def __repr__(self):
        return 'Boss: {0.name} {0.last_name}'.format(self)


class Worker(metaclass=MetaPerson):
    def __init__(self, organization, *args, **kwargs):
        self.organization = organization

    def __repr__(self):
        return 'Worker: {0.name} {0.last_name}'.format(self)


class Organization(metaclass=MetaOrganization):

    def __init__(self, name):
        self.name = name
        self.boss = None
        self.members = list()

    def __repr__(self):
        return 'Organizacion: {}'.format(self.name)

    def pick_one_worker(self):
        return random.choice(self.members)


if __name__ == '__main__':
    salo = Organization('Salo')
    print(salo)
    salo()
    print()
    sola = Organization('Sola')
    print(sola)
    sola()

    z = Organization('Salo')
    print("Nombres utilizados {}".format(Organization.instances.keys()))
    print()

    jefe_salo = Boss(salo)
    jefe_sola = Worker(sola)
    jefe_sola = Boss(sola)
    print()

    for i in range(3):
        w = Worker(salo)
        jefe_salo.add_member(w)
        w.to_work()
    salo.pick_one_worker().to_work()
    jefe_salo.order()
    print()
    for i in range(2):
        jefe_sola.add_member(Worker(sola))
    sola.pick_one_worker().to_work()
    jefe_sola.order()

    new_jefe_salo = Boss(salo)

    print('--'*50)
    salo()
    salo.see_members()
    print('--'*50)
    sola()
    sola.see_members()
    print('--'*50)

Organizacion: Salo
Nombre de la Organizacion: Salo
Jefe: None
Numero de empleados: 0


Organizacion: Sola
Nombre de la Organizacion: Sola
Jefe: None
Numero de empleados: 0

Nombres utilizados dict_keys(['Salo', 'Sola'])


Estoy trabajando!
Estoy trabajando!
Estoy trabajando!
Estoy trabajando!
Estoy mandando!

Estoy trabajando!
Estoy mandando!
----------------------------------------------------------------------------------------------------
Nombre de la Organizacion: Salo
Jefe: Boss: Valentina Pérez
Numero de empleados: 3

Miembro numero 0: Worker: Francisca López

Miembro numero 1: Worker: Constanza García

Miembro numero 2: Worker: Benito Gómez

----------------------------------------------------------------------------------------------------
Nombre de la Organizacion: Sola
Jefe: Boss: Geronimo Martínez
Numero de empleados: 2

Miembro numero 0: Worker: Peter Romero

Miembro numero 1: Worker: Simon Suárez

----------------------------------------------------------------------------